<a href="https://colab.research.google.com/github/franckoj/pyspark/blob/main/movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import *
spark=SparkSession.builder.appName('Usecase5').getOrCreate()

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip

In [ ]:
schema_item = 'movie_id long, movie_title string,releasedate string, videoreleasedate long,IMDbURL string, unknown string , Action string, Adventure string, Animation string,Childrens string, Comedy string, Crime string, Documentary string, Drama string, Fantasy string,FilmNoir string, Horror string, Musical string, Mystery string, Romance string, SciFi string,Thriller string, War string, Western string'
df_item = spark.read.csv('/content/ml-100k/u.item',sep='|',schema=schema_item)\
.withColumn('year',regexp_extract(col('movie_title'),'(\d{4})',1))\
.select('movie_id','movie_title','year')

In [ ]:
joined.groupBy('unknown','Action','Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','FilmNoir','Horror','Musical','Mystery','Romance','SciFi','Thriller','War','Western').agg(mean('rating')).show(3)

+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+------------------+
|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|FilmNoir|Horror|Musical|Mystery|Romance|SciFi|Thriller|War|Western|       avg(rating)|
+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+------------------+
|      0|     1|        0|        0|        0|     1|    0|          0|    0|      0|       0|     0|      1|      0|      0|    0|       0|  0|      0|3.7347670250896057|
|      0|     0|        0|        0|        0|     1|    1|          0|    0|      1|       0|     0|      0|      0|      0|    0|       0|  0|      0| 3.193798449612403|
|      0|     0|        0|        0|        0|     1|    0|          0|    0|      0|       0|     0|      0|      1|      0|    0|       1|

In [ ]:
joined.groupBy('Comedy').agg(mean('rating')).show()

In [ ]:
 joined=df_item.join(df_data, df_data.item_id == df_item.movie_id)

In [ ]:
schema_data = 'user_id string, item_id long, rating int, timestamp bigint'
df_data = spark.read.csv('/content/ml-100k/u.data',sep='\t', schema=schema_data)\
.withColumn('date',from_unixtime('timestamp', format='yyyy-MM-dd'))\
.select('item_id','rating','date')

In [ ]:
rating_count = df_data.groupBy('item_id').agg(count('rating').alias('user_rating_count'))

In [ ]:
joined = df_item.join(rating_count, df_item.movie_id == rating_count.item_id).select('movie_title','year','user_rating_count')

In [ ]:
joined.show(3)

+--------+--------------------+----+-------+-------------+
|movie_id|         movie_title|year|item_id|count(rating)|
+--------+--------------------+----+-------+-------------+
|     474|Dr. Strangelove o...|1963|    474|          194|
|      29|Batman Forever (1...|1995|     29|          114|
|      26|Brothers McMullen...|1995|     26|           73|
+--------+--------------------+----+-------+-------------+
only showing top 3 rows



In [ ]:
joined.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- item_id: long (nullable = true)
 |-- count(rating): long (nullable = false)



In [ ]:
 decade_67 = joined.where((joined.year >= '1960') & (joined.year < '1970')).sort(desc('user_rating_count')).take(1)

In [ ]:
decade_78 = joined.where((joined.year >= '1970') & (joined.year < '1980')).sort(desc('user_rating_count')).take(1)

In [ ]:
decade_89 = joined.where((joined.year >= '1980') & (joined.year < '1990')).sort(desc('user_rating_count')).take(1)

In [ ]:
 decade_01 = joined.where((joined.year >= '2000') & (joined.year < '2010')).sort(desc('user_rating_count')).take(1)

In [ ]:
 decade_90 = joined.where((joined.year >= '1990') & (joined.year < '2000')).sort(desc('user_rating_count')).take(1)

In [ ]:
 
data = decade_90,decade_67,decade_78,decade_89,decade_90
df = spark.createDataFrame([decade_67,decade_78,decade_89,decade_90]).toDF("col1")
#df.write.format('text').save('movie_ratinss')

In [ ]:
df1.select(col('col1').split('col1',','))

TypeError: ignored

In [ ]:
split_col = split(df1['col1'], ',')
df2 = df1.withColumn('movie_title', split_col.getItem(0,)).select('movie_title')
df3 = df1.withColumn('year', split_col.getItem(1)).select('year')
df4 =df1.withColumn('count',split_col.getItem(2)).select('count')

In [ ]:
df2.join(df3).show()

+--------------------+-----+
|         movie_title| year|
+--------------------+-----+
|      {Psycho (1960)| 1960|
|      {Psycho (1960)| 1977|
|   {Star Wars (1977)| 1960|
|   {Star Wars (1977)| 1977|
|      {Psycho (1960)| 1983|
|      {Psycho (1960)| 1997|
|   {Star Wars (1977)| 1983|
|   {Star Wars (1977)| 1997|
|{Return of the Je...| 1960|
|{Return of the Je...| 1977|
|     {Contact (1997)| 1960|
|     {Contact (1997)| 1977|
|{Return of the Je...| 1983|
|{Return of the Je...| 1997|
|     {Contact (1997)| 1983|
|     {Contact (1997)| 1997|
+--------------------+-----+



In [ ]:
df1 = df.withColumn('col1', df.col1.cast("string"))

In [ ]:
decade_67

TypeError: ignored

In [ ]:
##df_data.select(df_data.date.cast('date').alias('new_date')).filter(col('new_date')=='').count()

In [ ]:
##df_data.where((df_data.date > '1979') | (df_data.date < '1980')).show(4)

+-------+------+----------+
|item_id|rating|      date|
+-------+------+----------+
|    242|     3|1997-12-04|
|    302|     3|1998-04-04|
|    377|     1|1997-11-07|
|     51|     2|1997-11-27|
+-------+------+----------+
only showing top 4 rows



In [ ]:
agg_data = df_data.groupby('item_id').agg(mean('rating').alias('average_rating'),\
                               min('rating').alias('lowest_rating'),\
                               max('rating').alias('highest_rating'))

In [ ]:
agg_data.show(5)

+-------+------------------+-------------+--------------+
|item_id|    average_rating|lowest_rating|highest_rating|
+-------+------------------+-------------+--------------+
|    474| 4.252577319587629|            2|             5|
|     29|2.6666666666666665|            1|             5|
|     26| 3.452054794520548|            1|             5|
|    964|3.3333333333333335|            1|             5|
|   1677|               3.0|            3|             3|
+-------+------------------+-------------+--------------+
only showing top 5 rows



In [ ]:
usecase_8 = agg_data.join(df_item, agg_data.item_id==df_item.movie_id)\
.select('movie_id','movie_title','year','average_rating','lowest_rating','highest_rating').sort('movie_id')

In [ ]:
usecase_8.repartition(1).write.option('header',True).csv('movie_ratingss')

In [ ]:
joined=df_data.join(df_item, df_data.item_id== df_item.movie_id).show()

+-------+------+----------+--------+--------------------+
|item_id|rating|      date|movie_id|         movie_title|
+-------+------+----------+--------+--------------------+
|    242|     3|1997-12-04|     242|        Kolya (1996)|
|    302|     3|1998-04-04|     302|L.A. Confidential...|
|    377|     1|1997-11-07|     377| Heavyweights (1994)|
|     51|     2|1997-11-27|      51|Legends of the Fa...|
|    346|     1|1998-02-02|     346| Jackie Brown (1997)|
|    474|     4|1998-01-07|     474|Dr. Strangelove o...|
|    265|     2|1997-12-03|     265|Hunt for Red Octo...|
|    465|     5|1998-04-03|     465|Jungle Book, The ...|
|    451|     3|1998-02-01|     451|       Grease (1978)|
|     86|     3|1997-12-31|      86|Remains of the Da...|
|    257|     2|1997-11-12|     257| Men in Black (1997)|
|   1014|     5|1997-11-17|    1014|Romy and Michele'...|
|    222|     5|1997-10-05|     222|Star Trek: First ...|
|     40|     3|1998-03-27|      40|To Wong Foo, Than...|
|     29|     

In [ ]:
joined.printSchema()

AttributeError: ignored

In [ ]:
joined.select('movie_id').count()

In [ ]:
df2.groupBy('itemid').agg(F.desc('itemid')).show()

In [ ]:
joined_df = df1.crossJoin(df2)

In [ ]:
joined_df.show(4)

172800000,315360000

In [ ]:
df1.join(df2, df1.movieid == df2.groupBy('itemid'), 'outer')

In [ ]:
joined_df.summary()

In [ ]:
joined_df.groupBy('itemid').agg(F.mean('rating').alias('user_ratings')).show()

In [ ]:
f=

In [ ]:
df4=df2.groupBy('itemid').agg(F.mean('rating').alias('user_ratings'))

In [ ]:
df5=df4.join(df3)

In [ ]:
dd=df5.select('itemid','user_ratings')

In [ ]:
dfdate=df5.withColumn('date',F.from_unixtime('timestamp','yyyy-MM-dd')).select('date')

In [ ]:
dd.join(dfdate).show()

In [ ]:
df6=df5.where((df5.timestamp >315532800) & (df5.timestamp<631152000))

In [ ]:
df6.withColumn('date',F.from_unixtime('timestamp','yyyy-MM-dd')).show()

In [ ]:
from_date ='1980-1-1'
to_date = '1990-1-1'
import datetime
#convert string to datetimeformat
f_date = datetime.datetime.strptime(from_date, "%Y-%m-%d")
t_date = datetime.datetime.strptime(to_date, "%Y-%m-%d")
  
#convert datetime to timestamp
  
ft_date = int(datetime.datetime.timestamp(f_date))
tt_date = int(datetime.datetime.timestamp(t_date))
print(ft_date)
print(tt_date)